# Load the libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# Load the data

In [3]:
data = pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# What is a Column Transformer?
- It is a class in scikit - learn that is used to create and apply seperate transformers for numerical and categorical data.
- It is used to improve the quality of machine learning code by making it more organized and easier to maintain.
- When you want to combine multiple pipelines into one then we go for the concept of Column Transformation.

# Create a Column Transformer to be applied on the data
#### Steps to create column transformer

1. Create a pipeline for all the numerical variables
2. Create a pipeline for the categorical variables
3. Create a column transformer for the above pipelines

# Create a numerical pipeline for Age and Fare columns

In [4]:
from sklearn.impute import SimpleImputer
si_num = SimpleImputer(strategy = "median")
si_num

SimpleImputer(strategy='median')

In [5]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms

MinMaxScaler()

In [6]:
from sklearn.pipeline import make_pipeline
num_pipe = make_pipeline(si_num, mms)
num_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('minmaxscaler', MinMaxScaler())])

# Create a pipeline for the Categorical Variables

In [7]:
from sklearn.impute import SimpleImputer
si_cat = SimpleImputer(strategy = "most_frequent")
si_cat

SimpleImputer(strategy='most_frequent')

In [8]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe

OneHotEncoder()

In [9]:
from sklearn.pipeline import make_pipeline
cat_pipe = make_pipeline(si_cat, ohe)
cat_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder', OneHotEncoder())])

In [10]:
num_features = ["Age", "Fare"]
cat_features = ["Sex", "Embarked"]

# From the num_pipe and cat_pipe create a column transformer

In [12]:
from sklearn.compose import ColumnTransformer
col_trans = ColumnTransformer(
                                        [("num_pipeline", num_pipe, num_features),
                                        ("cat pipeline", cat_pipe, cat_features)]
)
col_trans

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('minmaxscaler',
                                                  MinMaxScaler())]),
                                 ['Age', 'Fare']),
                                ('cat pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder())]),
                                 ['Sex', 'Embarked'])])

In [21]:
from sklearn.compose import make_column_transformer
col_trans = make_column_transformer(
                                        [(num_pipe, num_features),
                                        (cat_pipe, cat_features)]
)
col_trans

ColumnTransformer(transformers=[('tuple',
                                 (Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer(strategy='median')),
                                                  ('minmaxscaler',
                                                   MinMaxScaler())]),
                                  ['Age', 'Fare']),
                                 (Pipeline(steps=[('simpleimputer',
                                                   SimpleImputer(strategy='most_frequent')),
                                                  ('onehotencoder',
                                                   OneHotEncoder())]),
                                  ['Sex', 'Embarked']))])

In [15]:
data_piped = col_trans.fit_transform(data)
data_piped

array([[0.27117366, 0.01415106, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.4722292 , 0.13913574, 1.        , ..., 1.        , 0.        ,
        0.        ],
       [0.32143755, 0.01546857, 1.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.34656949, 0.04577135, 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.32143755, 0.0585561 , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.39683338, 0.01512699, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [16]:
pd.DataFrame(data_piped, columns = ["Age", "Fare", "Female", "Male", "C", "Q", "S"])

,Age,Fare,Female,Male,C,Q,S
0,0.271174,0.014151,0.0,1.0,0.0,0.0,1.0
1,0.472229,0.139136,1.0,0.0,1.0,0.0,0.0
2,0.321438,0.015469,1.0,0.0,0.0,0.0,1.0
3,0.434531,0.103644,1.0,0.0,0.0,0.0,1.0
4,0.434531,0.015713,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
886,0.334004,0.025374,0.0,1.0,0.0,0.0,1.0
887,0.233476,0.058556,1.0,0.0,0.0,0.0,1.0
888,0.346569,0.045771,1.0,0.0,0.0,0.0,1.0
889,0.321438,0.058556,0.0,1.0,1.0,0.0,0.0


In [17]:
test_data = pd.read_csv("test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
test_data_transformed = col_trans.fit_transform(test_data)
test_data_transformed

array([[0.4527232 , 0.01528158, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.61756561, 0.01366309, 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.8153765 , 0.01890874, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.50547277, 0.01415106, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.35381775, 0.01571255, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.35381775, 0.0436405 , 0.        , ..., 1.        , 0.        ,
        0.        ]])

In [19]:
pd.DataFrame(test_data_transformed, columns = ["Age", "Fare", "Female", "Male", "C", "Q", "S"])

,Age,Fare,Female,Male,C,Q,S
0,0.452723,0.015282,0.0,1.0,0.0,1.0,0.0
1,0.617566,0.013663,1.0,0.0,0.0,0.0,1.0
2,0.815377,0.018909,0.0,1.0,0.0,1.0,0.0
3,0.353818,0.016908,0.0,1.0,0.0,0.0,1.0
4,0.287881,0.023984,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...
413,0.353818,0.015713,0.0,1.0,0.0,0.0,1.0
414,0.512066,0.212559,1.0,0.0,1.0,0.0,0.0
415,0.505473,0.014151,0.0,1.0,0.0,0.0,1.0
416,0.353818,0.015713,0.0,1.0,0.0,0.0,1.0
